In [5]:
# Modules we will use
import numpy as np
import pygrib
import s3fs
import os
from datetime import timedelta, date

# Range of dates to iterate over, this is the range of dates covered in the GEFS reanalyis available on AWS
startDate = date(2000, 1, 1)
#endDate = date(2019, 2, 18)
endDate = date(2000, 1, 6)
   
# Find the directory we're working with    
directory = os.getcwd()

# Predetermined pixel indicies for Alta, lines below can be uncommented and used to find the indicies for other locations
i,j = 198,993
desiredLat = 40.57
desiredLon = -111.67
#grbs = pygrib.open('exampleGefsFile.grib2')
#desiredLon = 360 + desiredLon
#lats, lons = grbs[1].latlons()
#a = abs(lats - float(desiredLat)) + abs(lons - float(desiredLon))
#i,j = np.unravel_index(a.argmin(), a.shape)

# Use anonymous credentials to access the public data on NOAA's AWS
fs = s3fs.S3FileSystem(anon=True)

# Iterate through the days
delta = timedelta(days=1)
while startDate <= endDate:
    
    # Start the URL for the files to download
    bucketUrl = 'noaa-gefs-retrospective/GEFSv12/reforecast/'
    year = str(startDate.year)
    month = str(startDate.month).zfill(2)
    day = str(startDate.day).zfill(2)
    hour = '00'
    date = year + month + day + hour

    # Make sure we haven't already extracted this date
    f = open('gefsReanalysis.csv', 'r')
    contents = f.read()
    f.close()
    if contents.find(date) != -1:
        startDate += delta
        continue
    
    print('working on ' + str(startDate))
    
    # Finish the file URLs
    ensMember = 'c00'
    range = 'Days:1-10/'
    fileSuffix = '_' + ensMember + '.grib2'
    uFile = 'ugrd_hgt_' + date + fileSuffix
    vFile = 'vgrd_hgt_' + date + fileSuffix
    gustFile = 'gust_sfc_' + date + fileSuffix

    # Download the files
    fs.get(bucketUrl + year + '/' + date + '/' + ensMember + '/' + range + uFile, uFile)
    fs.get(bucketUrl + year + '/' + date + '/' + ensMember + '/' + range + vFile, vFile)
    fs.get(bucketUrl + year + '/' + date + '/' + ensMember + '/' + range + gustFile, gustFile)

    # Use pygrib to open the grib2 files
    ugrbs = pygrib.open(uFile)
    vgrbs = pygrib.open(vFile)
    gustGrbs = pygrib.open(gustFile)

    # Initialize the string to hold the results
    output = date + ','

    # Loop through the forecast hours contained in the U and V files
    index = 1
    for ugrb in ugrbs:
        strName = str(ugrb)
        # Multiple heights of wind are contained here, need the 10m winds
        if 'level 10 m' in strName:
            # Check forecast hour, we're only running through 48 hours
            fcstTime = strName.find('fcst time ') + len('fcst time ')
            fhr = strName[fcstTime:strName.find('hrs',fcstTime)].strip(' ')
            if int(fhr) > 48:
                break
            
            # Grab U and V grids
            uArray = ugrb.values
            uArray = uArray[i][j]
            vArray = vgrbs[index].values
            vArray = vArray[i][j]
            
            # Iterate through gust grib2 file to the same fhr
            index2 = 0
            for gustgrb in gustGrbs:
                strName2 = str(gustgrb)
                fcstTime2 = strName2.find('fcst time ') + len('fcst time ')
                fhr2 = strName2[fcstTime2:strName2.find('hrs',fcstTime2)].strip(' ')
                if fhr.strip(' ') == fhr2.strip(' '):
                    gust = gustgrb.values
                    gust = gust[i][j]
                    break
            
            # Assemble comma-separated line with the fhr followed by u,v,gust
            output = output + fhr + ',' + '%.2f' % uArray + ',' + '%.2f' % vArray + ',' + '%.2f' % gust + ','
        index += 1

    # Format the output line as desired
    output = output[:-1]
    output = output + '\n'
    
    # Close the pygrib instances and remove the downloaded files
    ugrbs.close()
    vgrbs.close()
    gustGrbs.close()
    os.remove(directory + '/' + uFile)
    os.remove(directory + '/' + vFile)
    os.remove(directory + '/' + gustFile)

    # Append to the output file
    f = open('gefsReanalysis.csv', 'a')
    f.write(output)
    f.close()
    
    # Iterate to the next date
    startDate += delta


working on 2000-01-04
working on 2000-01-05
working on 2000-01-06
